## Branching Pipelines

In [1]:
import apache_beam as beam
import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner

In [2]:
def split_row(e: str):
    return e.split(",")

In [3]:
p = beam.Pipeline(InteractiveRunner())

input_collection = (
    p
    | "Read from file" >> beam.io.ReadFromText("dept_data.txt")
    | "Split by comma" >> beam.Map(split_row)
)

## branching
accounts_count = (
    input_collection
    | "Filter by account" >> beam.Filter(lambda r: r[3] == "Accounts")
    | "Pair each account employee with 1" >> beam.Map(lambda e: (e[1], 1))
    | "Count by account employee" >> beam.CombinePerKey(sum)
)

hr_count = (
    input_collection
    | "Filter by hr" >> beam.Filter(lambda r: r[3] == "HR")
    | "Pair each hr employee with 1" >> beam.Map(lambda e: (e[1], 1))
    | "Count by hr employee" >> beam.CombinePerKey(sum)
)

## flatten - union
output = (
    (accounts_count, hr_count)
    | "Combinding records" >> beam.Flatten()
)

ib.show(output, n=5)

In [4]:
ib.show_graph(p)

/usr/bin/dot
